In [1]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import os
import json
from dotenv import load_dotenv
from tqdm import tqdm_notebook as tqdm

# create a webdriver instance
# latest headless options: https://stackoverflow.com/questions/46753393/how-to-run-headless-firefox-with-selenium-in-python
options = Options()
# options.add_argument("-headless")
options.add_argument("-privatebrowsing")

driver = webdriver.Firefox(options=options)
# print("Headless Firefox Initialized")

# Define job and location search keywords
job_search_keyword = 'ai+engineer'
# Define Locations of Interest
location_search_keyword = "Singapore"

# Finding location, position, radius=35 miles, sort by date and starting page
# paginaton_url = 'https://www.indeed.com/jobs?q={}&l={}&radius=35&filter=0&sort=date&start={}'
base_url = f"https://sg.indeed.com/jobs?q={job_search_keyword}&l={location_search_keyword}"
# https://sg.indeed.com/jobs?q=data+engineer&l=Singapore
driver.get(base_url)
 
# waiting for the page to load
time.sleep(3)

In [2]:
job_count_text=driver.find_element(By.CLASS_NAME,'jobsearch-JobCountAndSortPane-jobCount').text
job_count=int(job_count_text.split(" ")[0])
assert job_count > 0
job_count

753

In [3]:
first_page = driver.page_source

soup = BeautifulSoup(first_page, 'lxml')
num_results_div = soup.find("div", attrs={"class": "jobsearch-JobCountAndSortPane-jobCount"})
num_results_text = num_results_div.span.get_text(strip=True)
int_num_results = int(num_results_text.split()[0])
assert isinstance(int_num_results, int) and int_num_results > 0

int_num_results

753

In [4]:
import hashlib

# Create a SHA-1 hash object
sha1_hash = hashlib.sha1()

# # Update the hash object with the bytes-like object (encoded string)
# sha1_hash.update(input_string.encode('utf-8'))

# # Get the hexadecimal representation of the hash
# hashed_result = sha1_hash.hexdigest()

job_desc_id = {}

In [5]:
job_page = driver.find_element(By.ID,"mosaic-jobResults")
jobs = job_page.find_elements(By.CLASS_NAME,"job_seen_beacon") # return a list
# print(len(jobs))

for jj in jobs:
    driver.execute_script("arguments[0].scrollIntoView();", jj)
    jj.click()
    time.sleep(3)
    first_page = driver.page_source

    soup = BeautifulSoup(first_page, 'lxml')
    # Find all 'a' elements that have a 'span' containing the 'title' attribute
    job_title = soup.find('h2', class_='jobsearch-JobInfoHeader-title').get_text()
    job_desc = soup.find('div', {'id': "jobDescriptionText"}).get_text()

    job_title_p1 = ''.join([i if ord(i) < 128 else ' ' for i in job_title])
    # convert all to lowercase
    job_title_p2 = job_title_p1.lower()
    # merge consecutive spaces
    job_title_p3 = " ".join(job_title_p2.split())
    # merge consecutive spaces
    job_title_p4 = r"\n".join(job_title_p3.split(r"\n"))
    job_title_p5 = job_title_p4.replace(" - job post", "")
    print(job_title_p5)

    job_desc_p1 = ''.join([i if ord(i) < 128 else ' ' for i in job_desc])
    # convert all to lowercase
    job_desc_p2 = job_desc_p1.lower()
    # merge consecutive spaces
    job_desc_p3 = " ".join(job_desc_p2.split())
    # merge consecutive spaces
    job_desc_p4 = r"\n".join(job_desc_p3.split(r"\n"))

    # # Update the hash object with the bytes-like object (encoded string)
    sha1_hash.update(job_desc_p4.encode('utf-8'))
    # # Get the hexadecimal representation of the hash
    hashed_result = sha1_hash.hexdigest()

    job_desc_id[hashed_result] = {
        "job_title": job_title_p5,
        "job_desc": job_desc_p4
    }


software engineer (ai verify)
senior ai engineer / ai engineer (innovation) for ai singapore
ai engineer (natural language processing)-ai singapore
ai engineer for ai singapore (research)
research engineer (ai speech-to-text engine)1
research engineer (robust ai)
research engineer (carbon reduction)
wallenberg - ntu presidential postdoctoral fellow
senior research engineer i (computer science)
ai engineer
ai/ nlp engineer
deputy director, software engineering & delivery lead
ai/ml - nlp engineer
ai algorithm engineer (biometrics)
ai engineer (generalist)


In [6]:
for i in range(1, 41, 1):
    base_url = f"https://sg.indeed.com/jobs?q={job_search_keyword}&l={location_search_keyword}&start={i*10}"
    # https://sg.indeed.com/jobs?q=data+engineer&l=Singapore
    driver.get(base_url)
    
    # waiting for the page to load
    time.sleep(3)

    # pagination = driver.find_element(By.XPATH,f"/html/body/main/div/div[1]/div/div[5]/div[1]/nav/ul/li[{i+1}]")
    # time.sleep(3)
    # driver.execute_script("arguments[0].scrollIntoView();", pagination)
    # pagination.click()
    # time.sleep(3)

    test_page = driver.page_source
    soup = BeautifulSoup(test_page, 'lxml')
    google_button = soup.find_all('button', {'class': "icl-CloseButton icl-Card-close"})

    if len(google_button) == 1:
        googleButton = driver.find_element(By.XPATH,"/html/body/div[3]/div/div/div[1]/button")
        time.sleep(1)
        googleButton.click()

        popupButton = driver.find_element(By.XPATH,"/html/body/div[4]/div[1]/div[1]/button")
        time.sleep(1)
        popupButton.click()


    job_page = driver.find_element(By.ID,"mosaic-jobResults")
    jobs = job_page.find_elements(By.CLASS_NAME,"job_seen_beacon") # return a list
    # print(len(jobs))

    for jj in jobs:
        driver.execute_script("arguments[0].scrollIntoView();", jj)
        jj.click()
        time.sleep(3)
        page = driver.page_source

        soup = BeautifulSoup(page, 'lxml')
        # Find all 'a' elements that have a 'span' containing the 'title' attribute
        job_title = soup.find('h2', class_='jobsearch-JobInfoHeader-title').get_text()
        job_desc = soup.find('div', {'id': "jobDescriptionText"}).get_text()

        job_title_p1 = ''.join([i if ord(i) < 128 else ' ' for i in job_title])
        # convert all to lowercase
        job_title_p2 = job_title_p1.lower()
        # merge consecutive spaces
        job_title_p3 = " ".join(job_title_p2.split())
        # merge consecutive spaces
        job_title_p4 = r"\n".join(job_title_p3.split(r"\n"))
        job_title_p5 = job_title_p4.replace(" - job post", "")
        print(job_title_p5)

        job_desc_p1 = ''.join([i if ord(i) < 128 else ' ' for i in job_desc])
        # convert all to lowercase
        job_desc_p2 = job_desc_p1.lower()
        # merge consecutive spaces
        job_desc_p3 = " ".join(job_desc_p2.split())
        # merge consecutive spaces
        job_desc_p4 = r"\n".join(job_desc_p3.split(r"\n"))

        # # Update the hash object with the bytes-like object (encoded string)
        sha1_hash.update(job_desc_p4.encode('utf-8'))
        # # Get the hexadecimal representation of the hash
        hashed_result = sha1_hash.hexdigest()

        job_desc_id[hashed_result] = {
            "job_title": job_title_p5,
            "job_desc": job_desc_p4
        }

ai engineer
ai engineer (generalist)
artificial intelligence ai software engineer
senior/research engineer (ai/ml), computing and intelligence, ihpc
[keppel internship programme 2024] intern, data engineer (may - aug/jun - dec 2024)
deputy director, software engineering & delivery lead
ai engineer
ai engineer
ai engineer
hewlett packard labs - ai/ml software engineer
customer engineer, ai search, google cloud
ai engineer - marketplace intelligence and data
nlp algorithm engineer - marketplace intelligence and data (campus recruitment 2024)
robotics engineer intern, singapore office
research engineer (engineering/ ai / machine learning) - wp3
ai engineer
research engineer (computer vision), trust and safety
machine learning engineer intern (trust & safety) - 2024
research engineer (computer vision), intelligent creation - 2024 start
deputy director, software engineering & delivery lead
algorithm engineer - chatbot (campus recruitment 2024)
ai engineer
machine learning engineer
ai engine

In [7]:
print(len(job_desc_id))

609


In [8]:
output_json = "sample.json"
with open(output_json, "w") as outfile:
    json.dump(job_desc_id, outfile)

In [9]:
driver.quit()

In [ ]:
# pagination = driver.find_element(By.XPATH,"/html/body/main/div/div[1]/div/div[5]/div[1]/nav/ul/li[2]")
# time.sleep(3)
# driver.execute_script("arguments[0].scrollIntoView();", pagination)
# pagination.click()

In [ ]:
# time.sleep(3)
# test_page = driver.page_source
# soup = BeautifulSoup(test_page, 'lxml')
# google_button = soup.find_all('button', {'class': "icl-CloseButton icl-Card-close"})

# if len(google_button) == 1:
#     googleButton = driver.find_element(By.XPATH,"/html/body/div[3]/div/div/div[1]/button")
#     time.sleep(3)
#     googleButton.click()

#     popupButton = driver.find_element(By.XPATH,"/html/body/div[4]/div[1]/div[1]/button")
#     time.sleep(3)
#     popupButton.click()

In [ ]:
# job_page = driver.find_element(By.ID,"mosaic-jobResults")
# jobs = job_page.find_elements(By.CLASS_NAME,"job_seen_beacon") # return a list
# # print(len(jobs))

# for jj in jobs:
#     driver.execute_script("arguments[0].scrollIntoView();", jj)
#     jj.click()
#     time.sleep(3)
#     first_page = driver.page_source

#     soup = BeautifulSoup(first_page, 'lxml')
#     # Find all 'a' elements that have a 'span' containing the 'title' attribute
#     job_title = soup.find('h2', class_='jobsearch-JobInfoHeader-title')
#     job_desc = soup.find('div', {'id': "jobDescriptionText"})
#     print(job_title.get_text())
#     print(job_desc.get_text())

In [ ]:
# job_results = soup.find("div", attrs={"class": "mosaic-zone", "id": "mosaic-jobResults"})
# with open("first-page.html", "w", encoding='utf-8') as file:
#     file.write(str(job_results))